# Project_One_이상봉

## 주제와 목적
최근 북미, 유럽, 일본, 그외 지역에서의 게임 판매량 데이터를 분석하고  
분석을 바탕으로 다음 분기에 게임 설계 방향을 정하는것이다    
정확한 분석을 위해서는 게임의 판매량에 영향을 미치는요인이 무엇인지 알아내야한다.  

## 우선 분석과정을 4개의 작은 단위로 나눠볼 수 있다
1. 지역별 선호하는 장르가 다른가?
2. 연도별 트랜드가 있는가?
3. 플랫폼에따른 판매량의 차이가 있는가?
4. 판매량이 가장 높은 게임들은 무슨 특징을 가지고 있는가?

이 분석과정을 통해, **"잘 팔리는"** 게임을 만들기위해 어떤 부분을 우선시 해야하는지를 판단해야 한다


분석을 시작하기전에, 데이터를 좀더 깔끔하게 다듬는 전처리 작업을 진행하였다  
# 전처리작업(EDA)
우선 데이터내의 수치를 일관성있게 통일하기위해 전처리작업을 진행하였다  
전처리작업이란, 숫자의단위나 알파벳의 대소문자의 통일, 알수없는값이나 비어있는값을 처리하여 데이터의 오류를 줄이는 과정이다.

In [237]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, f_oneway
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/AI_bootcamp/vgames2.csv', encoding='CP949')

# 인덱스제거
df = df.drop(['Unnamed: 0','Name','Publisher'], axis=1)

In [238]:
# 데이터 전처리하기


# 판매량을 (0.00) 단위로 통일하고 float으로 변경하는 함수 선언
def unification(string):
    # K가 있다면 지우고 천 곱하고 백만으로 나누기
    if ('K' in string) == True:
        # 만약 0K 라면 0.001로 바로바꾸기
        if string == '0K':
            return float(0.001)
        else:
            remove_k = string.replace('K','')
            return float(remove_k)*0.001
    # M이 있다면 지우기
    elif ('M' in string) == True:
        # 만약 OM 이라면 1.0으로 바꾸기
        if string == '0M':
            return float(1.0)
        else:
            remove_m = string.replace('M','')
            return float(remove_m)
    # 둘다없다면 타입만바꾸기
    else:
        return float(string)

# 판매량 칼럽의 단위통일시키기
df['NA_Sales'] = df['NA_Sales'].apply(unification)
df['JP_Sales'] = df['JP_Sales'].apply(unification)
df['EU_Sales'] = df['EU_Sales'].apply(unification)
df['Other_Sales'] = df['Other_Sales'].apply(unification)

# datatype이 object->float으로 되었는지 확인
df.dtypes

Platform        object
Year           float64
Genre           object
NA_Sales       float64
EU_Sales       float64
JP_Sales       float64
Other_Sales    float64
dtype: object

In [239]:
# Year 칼럼 네자리로 통일시키기
def make_year(year):
    if year < 21:
        return float(year+2000)
    elif year > 80 and year <= 99:
        return float(year+1900)
    else:
        return float(year)

df['Year'] = df['Year'].apply(make_year)

In [240]:
# 결측치 위치 확인하기
# 이값은 후에 다시 다루기
df.isna().sum()

Platform         0
Year           271
Genre           50
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

전처리가 끝난 데이터로 분석해보자  


# 1. 지역별 선호하는 장르가 다른가?  
다시말해  
**"장르의 지역별 판매량이 서로 비슷한가가?"**  
**"레이싱게임이 RPG게임보다  잘 팔리는가,  
아니면 미국에서만 잘팔렸는가?"**  
정도로 이해할 수 있다  

우선 장르들이 각각의 나라에서의 총판매량을 알아야한다  
그 후 각 장르의 총판매량이 얼마나 비슷한지를 확인하여
비슷하다면, 선호하는 장르가 비슷하다  
비슷하지않다면, 선호하는 장르다 다르다  
고 결론내릴 수 있다


In [241]:
# 장르와 판매량만 포함하는 df2 선언후 결측치 처리
df2 = df
df2 = df2.drop(['Year','Platform'],axis=1)
df2 = df2.dropna()
df2.isna().sum()

Genre          0
NA_Sales       0
EU_Sales       0
JP_Sales       0
Other_Sales    0
dtype: int64

In [262]:
# 각 지역별 판매된장르의 평균을 나타내는 데이터프레임 ge_lo 선언
ge_lo = df2.groupby('Genre').mean().T
ge_lo

Genre,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
NA_Sales,0.265904,0.084891,0.263518,0.236479,0.505023,0.213410,0.289776,0.218167,0.445658,0.216140,0.291980,0.109862
EU_Sales,0.159807,0.054728,0.121870,0.127085,0.228066,0.089502,0.192358,0.126489,0.239336,0.131044,0.161250,0.069590
JP_Sales,0.051410,0.042152,0.105483,0.068785,0.151195,0.101853,0.051013,0.232606,0.032327,0.075958,0.062956,0.074207
Other_Sales,0.058838,0.015480,0.045621,0.043279,0.061692,0.023394,0.062833,0.041784,0.080774,0.039865,0.060091,0.024065


In [263]:
# 각 지역별 장르의 분포를 비교하는 F-test 실시
ge_loc = f_oneway(ge_lo['Action'],ge_lo['Adventure'],ge_lo['Fighting'],ge_lo['Misc'],ge_lo['Platform'],ge_lo['Puzzle'],ge_lo['Racing'],ge_lo['Role-Playing'],ge_lo['Shooter'],ge_lo['Simulation'],ge_lo['Sports'],ge_lo['Strategy'])
print("P-value :", ge_loc.pvalue)

print("""
여기서의 귀무가설은 "장르별 판매량은 나라와 관련이없다" 이다

p-value의 값은 0.59 이므로 귀무가설을 기각할 수 없다
(장르별 판매량은 나라와 관련이없다)
즉 나라별 장르에대한 선호도는 다르지않다
""")

P-value : 0.5922861885716234

여기서의 귀무가설은 "장르별 판매량은 나라와 관련이없다" 이다

p-value의 값은 0.054 이므로 귀무가설을 기각할 수 없다
(장르별 판매량은 나라와 관련이없다)
즉 나라별 장르에대한 선호도는 다르지않다



# 2. 연도별 게임의 트랜드가 있을까?

"연도별 출시된 장르의 갯수가 다를까?"  
"2000년과 2002년에 출시된 RPG 게임의 수가 다를까?"  
정도로 생각해볼 수 있다.

연도별 출시된 게임들의 장르 총 갯수를 구해야한다  

갯수의 분포도를 비교하여 연도와 장르의 갯수가 연관이있는지 확인해보자



In [210]:
# 연도와 장르만 포함한 df3 선언후 결측치 처리
df3 = df[['Year','Genre']].dropna()
df3.isna().sum()

Year     0
Genre    0
dtype: int64

In [249]:
# 각 년도별 장르의 갯수를 포함하는 데이터프레임 ge_ye 선언
ge_ye = pd.crosstab(df3.Year,df3.Genre)

# 각 연도별 장르의 분포를 비교하는 F-test 실시
ge_year = f_oneway(ge_ye['Action'],ge_ye['Adventure'],ge_ye['Fighting'],ge_ye['Misc'],ge_ye['Platform'],ge_ye['Puzzle'],ge_ye['Racing'],ge_ye['Role-Playing'],ge_ye['Shooter'],ge_ye['Simulation'],ge_ye['Sports'],ge_ye['Strategy'])
print("P-value :", ge_year.pvalue)

print("""
여기서의 귀무가설은 "장르별 트랜드는 연도와 관련이없다" 이다

p-value의 값은 0.05보다 매우 작은 값이므로 귀무가설은 기각된다
(장르별 트랜드는 연도와 관련이 있다)
즉 연도별 게임의 트랜드가 있다
""")

P-value : 5.988229153302235e-11



# 3. 플랫폼과 판매량은 관계가 있을까? 
# 4. 장르와 판매량은 관계가 있을까?



## 판매량과 다른 요소들의 관계를 비교해보자  
위의 분석들을 통해 다음과 같은 결론을 내릴 수 있다  

판매량 - 장르  
판매량 - 플랫폼  
판매량 - 

이 우선순위를 바탕으로  
최근 판매량이 많은 게임을 분석하여 적용시켜본다면  

목적에 부합하는  
"잘 팔리는 게임"을 만들 수 있는 가능성이 높다

# 5. 판매량이 많은 게임을 분석하고 시각화해보자

# 결론
~ 한 게임을 만드는것이 좋다